In [132]:
import numpy as np
import pandas as pd
import json
from datetime import timedelta
import requests

In [133]:
DATA_URL = "http://ec2-108-137-45-5.ap-southeast-3.compute.amazonaws.com/api/netmon/rfcdata/"

In [134]:
res = requests.get(DATA_URL)
wpms_json = res.json()

In [135]:
with open("data.json", "w") as jsonfile:
    json.dump(wpms_json,jsonfile)

In [136]:
df = pd.read_json("data.json")

In [137]:
df['device'] = df['device'].apply(lambda x: x['name'])

In [138]:
def calculate_seconds(dt):
    if not dt.minute % 10:
        return dt.second
    return (dt.minute % 10) * 60 + dt.second

In [139]:
df['timestamp'] = df['timestamp'].apply(lambda d: d - timedelta(seconds=calculate_seconds(d)))

In [140]:
data = df[['device','mode','timestamp','rtt', 'bb', 'bdp', 'rwnd', 'rcvbuf_actual', 'actual_thpt', 'tcp_efficiency']]

In [141]:
data = data[data['timestamp'] >= '2022-08-03']

In [142]:
groups = data.groupby(['device', 'mode'])

In [143]:
data

,device,mode,timestamp,rtt,bb,bdp,rwnd,rcvbuf_actual,actual_thpt,tcp_efficiency
2589,RPI Malvar,reverse,2022-08-03 00:00:00,0.558225,48994044,27349700,3924480,7848960,49610323,99.857153
2590,RPI Cuenca,reverse,2022-08-03 00:00:00,0.560384,49000493,27459092,3924480,7848960,49358354,100.000000
2591,RPI Malvar,forward,2022-08-03 00:00:00,0.559134,14065481,7864488,1962240,3924480,17147153,100.000000
2592,RPI Cuenca,forward,2022-08-03 00:00:00,0.560258,14217822,7965648,1962240,3924480,19778590,100.000000
2593,RPI Cuenca,reverse,2022-08-03 00:10:00,0.559160,48861608,27321457,3924480,7848960,49112990,99.476329
...,...,...,...,...,...,...,...,...,...,...
4330,RPI Malvar,forward,2022-08-06 15:00:00,0.567225,13596754,7712419,981120,1962240,8755975,99.977566
4331,RPI Cuenca,forward,2022-08-06 15:00:00,0.569892,14005159,7981428,1962240,3924480,10068796,100.000000
4332,RPI Malvar,reverse,2022-08-06 15:10:00,0.561546,20517657,11521608,1962240,3924480,279834,84.958358
4333,RPI Malvar,forward,2022-08-06 15:10:00,0.566342,13913040,7879539,1962240,3924480,17004940,100.000000


In [131]:
with open('wpms_data.csv', 'w') as wpmscsv:
    data.to_csv(wpmscsv)